In [1]:
!pip install opencv-python
!pip install mtcnn
!pip install tensorflow==2.2
!pip3 install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
#from model import Net
import torch
from torchvision import transforms
from mtcnn import MTCNN

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
#!pip install torch===1.7.0 torchvision===0.8.1 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
def load_checkpoint(filepath):
    '''
    function to load saved model
    '''
    checkpoint = torch.load(filepath, map_location='cpu')
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    # as we've to perform testing, we don't need backpropagation so setting 'requires_grad' as false
    for parameter in model.parameters():
        parameter.requires_grad = False
    # model.eval() ->  .eval() does not change any behaviour of gradient calculations , but are used to set specific layers
    #                  like dropout and batchnorm to evaluation mode i.e. dropout layer won't drop activations and 
    #                  batchnorm will use running estimates instead of batch statistics.
    return model.eval()

In [4]:
train_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [5]:
# Initializing file paths for both the models
filepath1 = '../Saved Models/Real-Time Corona Mask Detection Model.pth'
filepath2 = '../Saved Models/Facial Keypoints Model.pt'

In [6]:
# Loading the models for testing
loaded_model_mask = load_checkpoint(filepath1)
#net = Net()
#net.load_state_dict(torch.load(filepath2))
#for parameter in net.parameters():
#        parameter.requires_grad = False
#net.eval()
#loaded_model_landmark = net

C:\Users\91844\anaconda3\lib\site-packages\torch\serialization.py:656: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\91844\anaconda3\lib\site-packages\torch\serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\91844\anaconda3\lib\site-packages\torch\serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attr

In [7]:
# Using GPU if available else using CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [8]:
detector = MTCNN()

In [9]:
!pip install face-alignment
import face_alignment
from skimage import io
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D , flip_input=False , device='cpu')

In [ ]:
# Accessing the webcam
cap = cv2.VideoCapture(0)

font = cv2.FONT_HERSHEY_DUPLEX
font_scale = 0.8
thickness = 1
red = (0,0,255)
green = (0,255,0)
blue = (255,255,0)
yellow = (255,255,255)

while (cap.isOpened()):
    # getting the frame in 'frame' and a bool value in 'ret' which is true if a frame is returned
    ret, frame = cap.read()
    size = frame.shape
    if ret == True:
        # converting into grayscale for feature reduction and grayscale images are less computation intensive to operate on
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        col = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # detecting the faces in the frame returned, it will return the coords of bounding box along with its height and width
        result = detector.detect_faces(col)
        
        for box in result:
            x, y, w, h = box['box']
            keypoints = box['keypoints']
            #2D image points. If you change the image, you need to change vector
            x1 = keypoints['mouth_right'][0]
            x2 = keypoints['mouth_left'][0]
            y1 = keypoints['mouth_right'][1]
            y2 = keypoints['mouth_right'][1]
            import math
            # drawing the bounding box based on the coordinates provided by haar_cascade
            preds = fa.get_landmarks(frame)
            
            midpoint_x = (x1+x2)/2
            midpoint_y = (y1+y2)/2
            ######distance#####
            dist = math.sqrt(math.pow((x1-x2),2)+math.pow((y1-y2),2))
            chin = (midpoint_x-math.sqrt(midpoint_y) , midpoint_y+5+(math.sqrt(y1+y2)-math.sqrt(x1+x2)))
            chin = (preds[2][2][0],preds[2][2][1])
            #print(preds)
            image_points = np.array([
                            (keypoints['nose']),     # Nose tip
                            (chin),     # Chin
                            (keypoints['left_eye']),     # Left eye left corner
                            (keypoints['right_eye']),     # Right eye right corne
                            (keypoints['mouth_left']),     # Left Mouth corner
                            (keypoints['mouth_right'])      # Right mouth corner
                        ], dtype="double")
 
            # 3D model points.
            model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                           (0.0, -330.0, -65.0),  
                              #(0.0,0.0,0.0),# Chin65
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner                     
                        ])

            # Camera internals
            focal_length = size[1]
            center = (size[1]/2, size[0]/2)
            camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )
            print ("Camera Matrix :\n {0}".format(camera_matrix))

            import matplotlib.pyplot as plt
            dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
            # Project a 3D point (0, 0, 1000.0) onto the image plane.
            # We use this to draw a line sticking out of the nose
            (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
            p1 = ( int(image_points[0][0]), int(image_points[0][1]))
            p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
            
            
            
            
            
            
            
            cv2.rectangle(frame, (x,y), (x+w,y+h), 2)
            # cropping the portion of image covered by the bounding box
            cropped_img = frame[y:y+h+2, x:x+w+2]
            # using Python's PIL lib to read image back into RGB format
            pil_image = Image.fromarray(cropped_img, mode="RGB")
            pil_image = train_transforms(pil_image)
            image = pil_image.unsqueeze(0)
            # feeding the test cropped image into the model
            result = loaded_model_mask(image)
            img = np.array(image)
            img = img[:,0,:,:]
            img = img.reshape(img.shape[0], 1, img.shape[1], img.shape[2])
            #result_lm = loaded_model_landmark(torch.from_numpy(img))
            #result_lm = np.array(result_lm)
            #result_lm = result_lm*(0.19*h)
            #result_lm = result_lm.reshape(68,2)
            #result_lm[:,0] += x+(0.28*h)
            #result_lm[:,1] += y+(0.49*w)
            _, maximum = torch.max(result.data, 1)
            prediction = maximum.item()
            preds = fa.get_landmarks(frame)
            
            print(preds)
            # displaying results based on classification
            if prediction == 0:
                cv2.line(frame, p1, p2, (255,0,0), 2) 
                preds = fa.get_landmarks(frame)
                for i in preds:
                    for j in i:
                        cv2.circle(frame,(int(j[0]),int(j[1])), 1, yellow, 1)
                cv2.putText(frame, "Correctly Masked", (x-33,y-10), font, font_scale, green, thickness)
                cv2.rectangle(frame, (x, y), (x+w+2, y+h+2), green, 2)  # green colour rectangle if mask is worn correctly
            elif prediction == 1:
                cv2.line(frame, p1, p2, (255,0,0), 2) 
                for i in preds:
                    for j in i:
                        cv2.circle(frame,(int(j[0]),int(j[1])), 1, yellow, 1)
                cv2.putText(frame, "Unmasked", (x,y-10), font, font_scale, red, thickness)
                cv2.rectangle(frame, (x, y), (x+w+2, y+h+2), red, 2)   # red colour rectangle if mask is not being worn
            elif prediction == 2:
                cv2.line(frame, p1, p2, (255,0,0), 2) 
                preds = fa.get_landmarks(frame)
                for i in preds:
                    for j in i:
                        cv2.circle(frame,(j[0],j[1]), 1, yellow, 1)
                cv2.putText(frame, "Incorrectly Masked", (x-37,y-10), font, font_scale, blue, thickness)
                cv2.rectangle(frame, (x, y), (x+w+2, y+h+2), blue, 2)   # blue colour rectangle if mask is not worn correctly
        cv2.imshow('frame', frame)
        if (cv2.waitKey(1) & 0xFF) == ord('q'):  # press 'q' to exit
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()